In [ ]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import norm
from scipy.stats import t as t_dist
import numpy as np
from scipy.stats import ttest_ind
from copy import deepcopy
import statsmodels.api as sm
import numpy as np

In [ ]:
#load the cleaned data
raw_data = pd.read_csv('train_clean.csv')

In [ ]:
#group the data by visCountryId and destinationId
df_dict = {}
grouped = raw_data.groupby(['visCountryId','destinationId'])
for name, group in grouped:
    if group[group["position"] == 1].shape[0] >= 250:
        df_dict[name] = group

In [ ]:
#store the outcome of the first position for each search
pop_data = []
for i in df_dict.keys():
    group1 = []
    group2 = []
    group_df = df_dict[i].groupby('srch_id')
    for name, group in group_df:
        if group['random_bool'].iloc[0] == 0:
            group1.append(np.sum(group[group.position == 1]['booking_bool']))
        else:
            group2.append(np.sum(group[group.position == 1]['booking_bool']))
    pop_data.append([group1,group2])

In [ ]:
#calculate the true tau
true_tao = []
for i in range(len(pop_data)):
    tau1 = pop_data[i][0]
    tau2 = pop_data[i][1]
    true_tao.append(np.mean(tau1) - np.mean(tau2))
true_tao1 = np.array(true_tao)

In [ ]:
plt.hist(true_tao, color = '#8CA5EA' , bins=20)
plt.xlabel('True HTEs')
plt.ylabel("Number")
plt.savefig('true_ate_in_Expedia.png', dpi=300, bbox_inches='tight')


In [ ]:
#calculate the weight for each group
weight = []
for i in range(len(pop_data)):
    tau1 = pop_data[i][0]
    tau2 = pop_data[i][1]
    weight.append(len(tau1) + len(tau2))
weight = np.array(weight)
weight = weight/np.sum(weight)

In [ ]:
T = 1000
N_list = [0.05,0.1,0.15,0.2,0.25]
cost = np.zeros((len(N_list),3,T))
decision_right_rate = np.zeros((len(N_list),3,2,T))


K = true_tao1.shape[0]
tau_min = 0.1
true_tao = true_tao1 - tau_min
optimal_cost = np.sum(true_tao[np.argwhere(true_tao>0)]*weight[np.argwhere(true_tao>0)])

estimated_tau = np.zeros((len(N_list),T,K))
calculte_zeros = np.zeros((2,T))
alpha = 0.05
Z = norm.ppf( 1 - alpha/2)
for idx,ratio in enumerate(N_list):
    N_array = np.zeros(K)
    for k in range(K):
        N_array[k] = int((len(pop_data[k][0]) + len(pop_data[k][1]))*ratio)
        #N_array[k] = ratio
    for t in tqdm(range(T)):
        #hist_data = np.zeros((K,N,2))
        tao_hat = np.zeros(K)
        variance = np.zeros(K)
        #upper_bound_list = np.zeros(K)
        p_value_list = np.zeros(K)
        
        group_hist = []
        for k in range(K):
            group_1 = np.array(np.random.choice(pop_data[k][0],int(N_array[k]/2),replace=True))
            group_0 = np.array(np.random.choice(pop_data[k][1],int(N_array[k]) - int(N_array[k]/2),replace=True))
            group_hist.append([group_1,group_0])
            
            diff_mean = group_1.mean() - group_0.mean()
            if np.std(group_1) == 0 and np.std(group_0) == 0:
                p_value = 0
            else:
                t_stat, p_value = ttest_ind(group_1, group_0, equal_var = False) 
            tao_hat[k] = diff_mean
            p_value_list[k] = p_value
            variance[k] = N_array[k]*(group_1.var(ddof=1) / len(group_1) + group_0.var(ddof=1) / len(group_0))
               
    
        decision1 = np.intersect1d(np.argwhere(p_value_list<alpha), np.argwhere(tao_hat>tau_min))
            
        tao_0 = np.mean(tao_hat)
        numerator = np.mean(variance)
        denumerator = np.mean((tao_hat - tao_0)**2) - np.mean(variance/N_array)

        bayesian_tao = np.zeros(K)
        bayesian_beta = np.zeros(K)
        p_value_list_bayesian = np.ones(K)
        decision4 = []
        #bayesian_variance = np.zeros(K)
        for k in range(K):
            if denumerator <= 0:
                theta = 1
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = variance[k]/N_array[k]
                if posteri_mean > norm.ppf(0.975)*np.sqrt(posteri_var) and posteri_mean > tau_min:
                    decision4.append(k)
            else:
                bayesian_beta[k] = max(variance[k]/denumerator,0)
                theta = N_array[k]/(N_array[k]+bayesian_beta[k])
                posteri_mean = tao_hat[k]*theta + (1 - theta)*tao_0
                posteri_var = theta*variance[k]/N_array[k]
                if posteri_mean > norm.ppf(0.975)*np.sqrt(posteri_var) and posteri_mean > tau_min:
                    decision4.append(k)
        

            
        if tao_0 == 0 and denumerator == 0:
            beta = 0
        else:
            beta = numerator/denumerator + Z*np.sqrt(np.mean(N_array*variance))/tao_0
        beta = max(beta,0)
        
        theta_array = np.zeros(K)
        for k in range(K):
            theta_array[k] = N_array[k]/(beta + N_array[k])

        
        

        tao_shunken_hat = np.zeros(K)
        p_value_list_shrunken = np.zeros(K)
    
        for k in range(K):
            group_11 = theta_array[k]*group_hist[k][0] + (1-theta_array[k])*tao_0
            group_00 = theta_array[k]*group_hist[k][1]
            if np.std(group_11) == 0 and np.std(group_00) == 0:
                p_value = 0
            else:
                t_stat, p_value = ttest_ind(group_11, group_00, equal_var = False)
    
            diff_mean1 = group_11.mean() - group_00.mean()
            tao_shunken_hat[k] = diff_mean1
            p_value_list_shrunken[k] = p_value
        
        decision2 = np.intersect1d(np.argwhere(p_value_list_shrunken<alpha), np.argwhere(tao_shunken_hat>tau_min))
        #decision3 = np.argwhere(bayesian_tao>tau_min)
        cost[idx,0,t] = np.sum(true_tao[decision1]*weight[decision1])/optimal_cost
        cost[idx,1,t] = np.sum(true_tao[decision2]*weight[decision2])/optimal_cost
        cost[idx,2,t] = np.sum(true_tao[decision4]*weight[decision4])/optimal_cost
       
        negative_right1 = 0
        negative_right2 = 0
        negative_right3 = 0
        positive_right1 = 0
        positive_right2 = 0
        positive_right3 = 0
        for k in range(K):
            if true_tao[k] < 0 and k not in decision1:
                negative_right1 += 1
            if true_tao[k] < 0 and k not in decision2:
                negative_right2 += 1
            if true_tao[k] < 0 and k not in decision4:
                negative_right3 += 1
            if true_tao[k] > 0 and k in decision1:
                positive_right1 += 1
            if true_tao[k] > 0 and k in decision2:
                positive_right2 += 1
            if true_tao[k] > 0 and k in decision4:
                positive_right3 += 1
        decision_right_rate[idx,0,0,t] = positive_right1/(len(np.argwhere(true_tao>0)))
        decision_right_rate[idx,1,0,t] = positive_right2/(len(np.argwhere(true_tao>0)))
        decision_right_rate[idx,2,0,t] = positive_right3/(len(np.argwhere(true_tao>0)))
        decision_right_rate[idx,0,1,t] = negative_right1/(len(np.argwhere(true_tao<0)))
        decision_right_rate[idx,1,1,t] = negative_right2/(len(np.argwhere(true_tao<0)))
        decision_right_rate[idx,2,1,t] = negative_right3/(len(np.argwhere(true_tao<0)))

   